In [3]:
# %%bash
# pip install -U ray[default]==2.2.0
# pip install "pydantic<2"

In [2]:
import ray

ray.init(address="ray://raycluster-kuberay-head-svc.default.svc.cluster.local:10001")

ClientContext(dashboard_url='10.244.0.65:8265', python_version='3.8.13', ray_version='2.2.0', ray_commit='b6af0887ee5f2e460202133791ad941a41f15beb', protocol_version='2022-10-05', _num_clients=1, _context_to_restore=<ray.util.client._ClientContext object at 0x7fd1f0a316a0>)

In [3]:
print(ray.cluster_resources())

{'accelerator_type:G': 2.0, 'GPU': 2.0, 'object_store_memory': 778088448.0, 'memory': 3000000000.0, 'CPU': 2.0, 'node:10.244.0.66': 1.0, 'node:10.244.0.65': 1.0}


In [4]:
# Try Ray task
@ray.remote
def f(x):
    return x * x

futures = [f.remote(i) for i in range(4)]
print(ray.get(futures)) # [0, 1, 4, 9]

[0, 1, 4, 9]


In [5]:
# Try Ray actor
@ray.remote
class Counter(object):
    def __init__(self):
        self.n = 0

    def increment(self):
        self.n += 1

    def read(self):
        return self.n

counters = [Counter.remote() for i in range(4)]
[c.increment.remote() for c in counters]
futures = [c.read.remote() for c in counters]
print(ray.get(futures)) # [1, 1, 1, 1]

[1, 1, 1, 1]
(scheduler +18m35s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +18m35s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


### Access Ray dashboard

Run a port-forward from your local computer to the ray dashboard:

`kubectl port-forward raycluster-kuberay-head-862db 8265:8265 -n default`

Go to http://localhost:8265/ and you should see the Ray dashboard.

### Connect to Ray cluster from local machine

> The version of the Ray client must match the version of the Ray cluster (2.2.0).

```bash
conda create -n ray-env python=3.8.10
conda activate ray-env
pip install ray==2.2.0
pip install "pydantic<2"
```

Run a port forwarding to the Ray head service:

```bash
kubectl port-forward svc/raycluster-kuberay-head-svc 10001:10001 -n default
```

```python
import ray

ray.init(address="ray://localhost:10001")
print(ray.cluster_resources())
```